# Homework 04: Midway Check-in Final Project


In [1]:
# June Wagner
# Homework 04

import math
import numpy as np
import pandas as pd
import time
from datetime import datetime
from matplotlib import pyplot as plt

## 2. Data Collection
### 2.1 Data Description

In [2]:
# Write a short description of the dataset (200+ words). Think on: where data was collected from, how, whose data it is, what
# are the main features of the dataset, how old it is, how often it is updated... etc.

# Dataset: "Game Recommendations on Steam" - by Anton Kozyriev, last updated 13 December 2023 (accessed 29 April 2024).
# Update frequency: semi-monthly basis; last updated 5 months ago.
# https://www.kaggle.com/datasets/antonkozyriev/game-recommendations-on-steam
# All data was collected from Steam, an online games store / games library application run by Valve Corporation.
# https://store.steampowered.com/
#
# There are four files in total: games.csv, games_metadata.json, recommendations.csv, and users.csv. According to the dataset's
# webpage, these files represent over 41 million user recommendations of games on Steam; according to my exploration, there are
# some 50,000 games represented--some of which are only Downloadable Content (DLC) or musical Original Soundtrack (OST).
# 
# games.csv contains a list of games on Steam as they appear in a snapshot--even their discounts (a temporary variable).
#    Each entry has an app_id that is unique, a title (does not contain commas), a date it was released on Steam,
#    which operating systems it is available on (Windows, Mac, Linux), a rating (e.g. positive, mixed, overwhelmingly negative),
#    the ratio of positive ratings (e.g. 90), the number of user-submitted reviews, the price (before and after any discount),
#    and whether it is available on the Steam Deck (almost all are).
# 
# games_metadata.json is a metadata .json file that adds colorful information to each game entry, like developer-submitted
#    descriptions of their game(s) and selected genre tags for the games. Many games lack a description and/or a genre tag.
# 
# recommendations.csv: a many-to-many relation between games and users, with positive and negative (TRUE or FALSE) reviews.
#    Each review has a recorded app_id to identify the game being reviewed, and a review_id to identify the individual review.
#    Although "helpful" and "funny" are included metrics, they represent other-user engagement with a given review.
#    The reveiwer's user_id is included, and their number of hours played on the given game is also given (as low as 0, rarely).
#    A date is also included, which could be helpful for quantifying a more-recent review as worth more than a less-recent one.
#
# users.csv: a list of users, their number of products (games), and their number of reviews. Very few have 0 games or 0 reviews.
#
# According to the description on Kaggle, the dataset is built from information already accessible to the public, and all 
# users are assigned an anonymized user ID.

# ***All of this information is further expanded in 3.1.1

### 2.2 Dataset Download

In [3]:
# 2.2.1: Download the dataset to the folder data.

# Don't forget to do this, June :)
# ** Add data to the git ignore file, so it is not pushed to the repository (because it's not our data to upload/distribute)...

## 3. Data Preprocessing
### 3.1 Data Cleaning

In [4]:
# 3.1.1 Perform some initial data exploration to understand the dataset better.

# games.csv:
#    games.csv contains a list of games on Steam as they appear in a snapshot--even their discounts (a temporary variable)
#    app_id - a critical information point that identifies each entry to a unique number
#    title - much less important; some of these titles contain bizarre characters, especially around trademark symbols... badly translated into ascii?
#    date_release - release date in (m)m/(d)d/yyyy format. We might filter out entries beyond 2020 since that's where the RepresentMe database stops.
#    win - available on a Windows platform. 
#    mac - available on a Mac platform.
#    linux - available on a Linux platform. (Least common)
#       If a game is triple-negative (not win, mac, or linux) it seems to be an Soundtrack and/or Artbook.
#          * I should remove these entries and ignore their app_id in other .csv data files.
#
#    rating - 9 Categories, by this general rubric:
#           - Overwhelmingly Positive: 95%+ positive reviews; 500+ reviews
#           - Very Positive          : 85%+ positive reviews; 50+ reviews
#           - Positive               : 80%+ positive reviews; 10+ reviews
#           - Mostly Positive        : 70%+ positive reviews; 10+ reviews
#           - Mixed                  : 40%+ positive reviews; 10+ reviews
#           - Mostly Negative        : 20%+ positive reviews; 10+ reviews
#           - Negative               : 00%+ positive reviews; 10+ reviews
#           - Very Negative          : 00%+ positive reviews; 50+ reviews
#           - Overwhelmingly Negative: 00%+ positive reviews; 500+ reviews
#       If a game is Mostly Negative or worse, I should probably strike it from the model? We are trying to recommend good game experiences.
#    positive_ratio - the ratio of positive reviews, accurate to 2 digits (e.g. 94% or 11%).
#       I think that this is relatively redundant to the "rating", especially because "rating" includes a "confidence" (i.e. 'very')
#    user_reviews - the number of reviews a game has had (min: 10 max: 7494460). 
#       There are no games with fewer than 10 reviews here, and this is useful because a game needs 10+ reviews to qualify for "rating"
#       
#       * I think that 'rating' makes for a decent enough metric that it is okay to mostly ignore the positive_ratio and user_reviews,
#          especially since it is a useful product of those two and represents both positivity and confidence of a positive experience.
#
#    price_final - price after a discount is applied to the price_original.
#    price_original - the original price of the game, before any potential discount is applied.
#    discount - the percentage (min: 0%; max: 90%) discount on the game.
#        This is a mostly trivial section of the data, but it is useful to report price and/or discount as part of the game insofar as
#        an end user would like to know how much a game costs before following through with a purchase. I would consider this to be
#        a more decorative sort of data, not very important to what we're trying to accomplish here.
#    steam_deck - TRUE or FALSE: compatible with the Steam Deck gaming platform. There are exactly two games that are FALSE.

# games_metadata.json: a metadata .json file that adds information to each game entry.
#     app_id: a unique number assigned to each game, perfectly 1-for-1 to the game's entry in games.csv app_id.
#     description: flavor text that may or may not exist (i.e. some entries are null), as a description for the game.
#     tags: game genres as an array (e.g. Action, Adventure, Simulation, RTS, 2D, Rougelike); some entries are empty / null.
#        
#        * This could be useful information except that so many games don't have tags and/or a description! 
#        I think it is worth delivering this info to the end-user at least; it could be worth exploring a user's 
#        preference for genres if the many-to-many users.csv file doesn't deliver similarly important information.

# recommendations.csv: a many-to-many relation between games and users, with positive and negative (TRUE or FALSE) reviews.
#     app_id: a unique number assigned to each game, exactly the same as in games.csv and games_metadata.json ...
#     helpful: the number of other-user reactions to this review that indicate it is "helpful" (range: 0, 2 to 16163),
#        more than half of these entries are "0", but that doesn't necessarily mean these reviews are unhelpful.
#        Potentially just that this user review hasn't had many eyes on it.
#     funny: the number of other-user reactions to this review that indicate it is "funny" (range: 0, 2 to 8818),
#        more than 3/4 of these entries are "0". This might not be an especially useful metric except that it means that
#        other users have read and engaged with this review.
#     date: the date a review was published. This could be helpful, since more recent reviews are more relevant to the current
#        release of the game than older reviews.
#     is_recommended: TRUE or FALSE. "FALSE" (i.e. negative) is fewer than 1/10 of the reviews. A negative review is much less common
#        than a positive one. Are negative reviews useful for steering our decisions? Maybe even more than positive reviews?
#        After all, a negative review means a lot more than simply no review at all. A user must really go out of their way for it!
#        But they are much less common than positive reviews; positive reviews will likely form the majority basis for what we 
#        want to accomplish here.
#     hours: the number of hours a user has in a game, (range: 0 to 999.9; clearly truncating at 999.9 as so many users will have
#        more than 1000 hours); this can be seen as a "confidence" of the review. A review with 0 hours may be worth less than a
#        review at 999.9+ hours.
#     user_id: a number assigned to each unique user; this number can repeat--this means that that user has made multiple reviews.
#     review_id: a number assigned to each unique review.

# users.csv: a list of users, their number of products (games), and their number of reviews.
#     user_id: a number assigned to each unique user. Just as in recommendations.csv.
#     products: the number of games the user has (range: 0, 2 to 21112). Very few are 0.
#        I think that users that have "0" products actually 'used' to own one or more games, but now do not own any (refund?)
#        (Maybe confiscation for violation of Steam's Terms of Service?)
#     reviews: the number of reviews this user has produced (range: 0 to 1762); very few are 0.
#        I wonder if the users with "0" reviews actually created a review that was removed? Maybe for moderation reasons (profanity)?
#     

In [5]:
# 3.1.2 Address missing data, if any, by appropriately identifying and quantifying missing values in the dataset.

# By visual inspection (messing around in Excel) the data shows no null values except in the .json, where there are many 
# games with no description and/or tags (genre). The dataset does not have users with 1 product, games with <10 reviews, etc.,
# so it would appear that the dataset has already had some cleaning done to it and I won't be able to tell if some values
# were changed. I wonder if the lack of "1" as a datapoint in some columns is because it could be read as "TRUE" in some 
# programs. I assume that some--but not all--outliers are already dealt with.


# WIP: 

# See 3.1.5 for checking for further missing values.

In [6]:
# 3.1.3 Apply suitable techniques to handle missing values (e.g. imputation, exclusion) based on a logical approach.
# 3.1.4 Justify the chosen method in 50+ words.

# This dataset is already quite clean, as shown in 3.1.5
# While the games_metadata.json does have quite a few missing values, it is already not going to be relied on. If it could
# have at least one tag for (most) games, I might see about using imputation to classify the remainder as "untagged" and see
# if tags could be used to help recommend games, but in the current circumstances this doesn't make much sense.

# I suppose I am excluding information from the games_metadata.json altogether for its incompleteness. If it is used, it will
# be in order to deliver flavorful information to the end user about the generated recommendations.

In [7]:
# 3.1.5.a Checking for missing values with code. (games.csv)
games_df = pd.read_csv("games.csv")


games_df.info() # get a simple summary
games_df.isna().sum() # if all numbers are 0, there are no null values.



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50872 entries, 0 to 50871
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   app_id          50872 non-null  int64  
 1   title           50872 non-null  object 
 2   date_release    50872 non-null  object 
 3   win             50872 non-null  bool   
 4   mac             50872 non-null  bool   
 5   linux           50872 non-null  bool   
 6   rating          50872 non-null  object 
 7   positive_ratio  50872 non-null  int64  
 8   user_reviews    50872 non-null  int64  
 9   price_final     50872 non-null  float64
 10  price_original  50872 non-null  float64
 11  discount        50872 non-null  float64
 12  steam_deck      50872 non-null  bool   
dtypes: bool(4), float64(3), int64(3), object(3)
memory usage: 3.7+ MB


app_id            0
title             0
date_release      0
win               0
mac               0
linux             0
rating            0
positive_ratio    0
user_reviews      0
price_final       0
price_original    0
discount          0
steam_deck        0
dtype: int64

In [8]:
# 3.1.5.b Checking for missing values with code. (users.csv)
users_df = pd.read_csv("users.csv")

users_df.info() # get a simple summary
users_df.isna().sum() # if all numbers are 0, there are no null values.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14306064 entries, 0 to 14306063
Data columns (total 3 columns):
 #   Column    Dtype
---  ------    -----
 0   user_id   int64
 1   products  int64
 2   reviews   int64
dtypes: int64(3)
memory usage: 327.4 MB


user_id     0
products    0
reviews     0
dtype: int64

In [ ]:
# 3.1.5.c Checking for missing values with code. (recommendations.csv)   *** WARNING: THIS TAKES TIME! 2.2+ GB data into memory!
recom_df = pd.read_csv("recommendations.csv")

recom_df.info() # get a simple summary
recom_df.isna().sum() # if all numbers are 0, there are no null values.

In [ ]:
# 3.1.5.d Checking for missing values with code. (games_metadata.json)
gmeta_df = pd.read_json("games_metadata.json", lines = True)  # *** Throws ValueError for "trailing data" if not lines = True

gmeta_df.info() # get a simple summary
gmeta_df.isna().sum() # if all numbers are 0, there are no null values.

### 3.2 Outlier Detection

In [ ]:
# 3.2.a Check for outliers: games_df

# *** I want to use games_df as an example of finding outliers and excluding them.
#     This is already a very clean dataset, so removing datapoints shouldn't be done just because they skew distribution.
#     This is just how I would do it if I was going to :)

# Then, I will apply math (interquartile range * some threshold, like 1.5) to find specific outliers.
# Finally, I will deal with these outliers logically.
pd.options.display.float_format = '{:.0f}'.format # This option suppresses Pandas's scientific notation, allows 0 decimal.

# I will start with games_df
games_df.describe()

In [ ]:
# 3.2.a games_df outliers check continued

# We should ignore app_id. This might as well be a string.

# positive_ratio: I think there will be outliers below the 25th quartile. It might be worth excluding these poorly-rated games.
# user_reviews: There are absolutely outliers in user_reviews above the 75th quartile. I don't think its worth excluding 
#               games that have a ton of reviews; this just means that they're more popular, and we merely should make sure
#               that we don't necessarily count a very high number of reviews as much more valuable than the 75th quartile of
#               reviews (7494460 should be relatively similar to 206).
#
#               In fact, Steam uses the thresholds 10+, 50+, 500+ reviews to determine confidence of reviews.
#               Consider that the ratio of 500 to 206 is 2.43; this is a pretty reasonable tolerance (standard is 1.5). 
#               Maybe we make sure down the line that we treat 7494460 reviews the same as 500 reviews?
#               The way to do this would probably be to rely on the strings in
#               "rating": Overwhelmingly positive, Very positive, Positive, ..., Overwhelmingly negative.

# We will be ignoring price_final, price_original, and discount. These columns don't matter to us--we just want games' quality.

q1_posratio = games_df["positive_ratio"].quantile(0.25)
q3_posratio = games_df["positive_ratio"].quantile(0.75)
iqr_posratio = q3_posratio - q1_posratio  # 91 - 67 = 24
floor_posratio = q1_posratio - iqr_posratio * 1.5  # 1.5 is the threshold here. It is somewhat arbitrary.
#print(floor_posratio) # = 31
posratio_outliers = games_df[(games_df["positive_ratio"] < floor_posratio)] # If positive_ratio < 31, then exclude.

display(posratio_outliers)
print("As expected, we get a dataframe full of quite negatively-rated games. We will exclude these (Removal).")

In [ ]:
# 3.2.a games_df positive_ratio outliers dealt with, to illustrate an example.

display(games_df.drop(posratio_outliers.index).sort_values(by = "positive_ratio"))

print(f"With this removal, our games_df in progress would be without the {posratio_outliers.shape[0]} extremely negatively-rated outliers.")
print(f"However, this isn't actually a desirable outcome--these outliers aren't because of some measurement error!")
print(f"Rather, we should keep these poorly-rated games in because they're just another part of our data.")
print(f"If ultimately our work ends up recommending the 'outlier' negatively-rated games, then we know something is wrong.")

In [ ]:
# 3.2.a games_df outlier discussion conclusion

# Ultimately, we shouldn't be just removing these otherwise valid data points so that we get a more normal-looking distribution.
# But at least we have shown how it could be done :)


In [ ]:
# 3.2.b Now users_df gets described and examined for outliers.
display(users_df.describe())

plt.boxplot(users_df["products"])
plt.title("Products Boxplot")
plt.show()

plt.boxplot(users_df["reviews"])
plt.title("Reviews Boxplot")
plt.show()

In [ ]:
# 3.2.b users_df examination for outliers continued

# So we see that these are extremely skewed. The majority of users have 127 or fewer Steam products, and have made 3 or
# fewer reviews. Yet there are all these outliers with a huge amount of products or reviews!

# Thinking ahead, we might actually want to remove some of these outliers ... they are likely bots or mal-actor users who 
# intend to make tons of reviews for some potentially nefarious, selfish, or silly purpose. There are, for instance, review
# bots that rate games on whether or not they contain something in particular (e.g. a dog that can be pet). These are not
# useful to us, and removing them might make our dataset much more reflective of a more average user-- the true audience we 
# are trying to get at.

# Let's try another way of identifying the outliers: 3x STDev (if it falls beyond 3 standard deviations from the mean, it
# is an outlier).

ceil_reviews = users_df["reviews"].mean() + 3*users_df["reviews"].std() # ~26.8
# We don't bother with a lower bound; it would be a negative number in this case since the data is skewed so hard one direction.

reviews_outliers = users_df[(users_df["reviews"] > ceil_reviews)] # If reviews >= 27, then exclude

display(reviews_outliers.sort_values(by = "reviews"))

print(f"As anticipated, our outliers range 27 to 6045 reviews. The number of outliers actually represents <",end="")
print(f"{math.ceil(100*reviews_outliers.shape[0]/users_df.shape[0])}% of total reviews.")




In [ ]:
users_df_clean = users_df.drop(reviews_outliers.index).sort_values(by = "reviews")
display(users_df_clean)

print("This represents the vast majority of reviewers, and hopefully cuts out many non-human reviewers.")
print("I don't think it makes too much sense to also curtail by number of products a reviewer has.")


In [ ]:
# 3.2.c recom_df examination for outliers

recom_df.describe()   # Note: may take a long time.

In [ ]:
# 3.2.c continued
# Importantly, this information already is cut off: any user with >=1000 hours in a game is recorded as having ~1000 hours.

# Otherwise, the "helpful" and "funny" metrics don't seem to matter much. This is how much other users engage with a review.

# It might make sense to cut out the reviews with the least hours played, but even these are important: some games are extremely
# short, or some people may be simply unable to launch the game they've purchased.

# The other metrics are ID numbers, so no need to check for further outliers.

In [ ]:
# 3.2.d
# And finally, the metadata (games_metadata.json)
gmeta_df.describe()

print("Since there isn't a numerical value outside app_id, we won't bother further examining games_metadata.")

### 3.3 Data Quality and Consistency

In [ ]:
# 3.3.1 Ensured data quality by verifying data consistency and accuracy.

# We performed a check for missing values already. But if we're going to strike some entries in users_df, we should strike
# those users from the recom_df too.

display(reviews_outliers)
print("We will have to remove all entries with these user_id numbers from recom_df.\nThis amounts to",end=" ")
print(f"{reviews_outliers['reviews'].sum()} entries, so {recom_df.shape[0]-reviews_outliers['reviews'].sum()} will remain.")
display(recom_df)


In [ ]:
# 3.3.1 continued

# We remove all entries from recom_df that have a user_id contained in reviews_outliers:
recom_df_clean = recom_df[~recom_df["user_id"].isin(reviews_outliers["user_id"])]
display(recom_df_clean)

# Verify that these two numbers are the same: that means that we dropped exactly as many entries as we wanted to.
print(f"As expected, {recom_df.shape[0]-reviews_outliers['reviews'].sum()} = {recom_df_clean.shape[0]}") 

In [ ]:
# 3.3.2 Check for duplicate records or potential errors; correct them as needed.

# Duplicates are simple:
print(f"games_df contains {games_df.duplicated().sum()} duplicates.")
print(f"users_df_clean contains {users_df_clean.duplicated().sum()} duplicates.")
print(f"recom_df_clean contains {recom_df_clean.duplicated().sum()} duplicates.")  # Warning, takes a long time.

In [ ]:
# 3.3.3 If a transformation was made, what was it and why?

# We made sure that all the outlier reviewers that had made too many reviews in users_df were also struck from recom_df; this
# means that our products (users_df_clean and recom_df_clean) will not have orphaned user_id numbers that exist in one and not  
# the other.

### 3.4 Data Preparation for Model Training

In [ ]:
# It has become obvious at this point that the most important dataset we're working with is the recommendations.csv, and our
# version of it: recom_df_clean. 